## **Student's Autograder**
This is a project to calculate quiz and term scores from q&a pdf and response pdf.

Whats left: 
* Designing report card
* Exporting dataframe
* Editing pdf

Importing the required libraries 

In [68]:
import os
import pandas as pd
import numpy as np
import tabula
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader, PdfFileWriter
from PyPDF2.generic import DecodedStreamObject, EncodedStreamObject,ArrayObject

In [69]:
#debugging for notebooks current directory for reading files later.
import os
os.getcwd()

'C:\\Users\\506761\\OneDrive - Tata Steel Limited\\Documents\\Python Scripts'

### Reading response pdf

In [70]:
# POD22S1C31570014.pdf
# POD22S1C11550029.pdf
df = tabula.read_pdf('POD22S1C11550029.pdf',pages='all',multiple_tables=False)

In [71]:
#IIT M DIPLOMA QUIZ1 EXAM QPF1
# IIT M QUIZ 1 FOUNDATION DAD DIPLOMA QPD1.pdf
filename="IIT M QUIZ 1 FOUNDATION DAD DIPLOMA QPD1.pdf"

In [72]:
df[0].tail()

,Name,SHEKHAR MOHANTY HEMANT
250,640653386636,Unanswered
251,640653386637,Unanswered
252,640653386628,Unanswered
253,640653386629,Unanswered
254,640653386630,Unanswered


### Variables initialisation

In [73]:
my_question="Question ID"
my_answer="Options Selected"
unanswered="Unanswered"
file_qid="Question Id"

In [74]:
df[0].iloc[4].values

array(['Question ID', 'Options Selected'], dtype=object)

In [75]:
df[0].head()

,Name,SHEKHAR MOHANTY HEMANT
0,QP Set,2022 Oct: IIT M QUIZ 1 FOUNDATION DAD DIPLOMA ...
1,Test Center Name,iON Digital Zone iDZ Jamshedpur
2,Shift,FN
3,Roll No,POD22S1C11550029
4,Question ID,Options Selected


In [76]:
df1=df[0].replace(unanswered,np.nan,regex=True)

Students response for exam dataframe

In [77]:
myans=df1.dropna()
myans.columns=myans.iloc[4].values
myans=myans[5:]

myans[my_question]=myans[my_question].apply(int)
myans=myans.sort_values(my_question)
print(len(myans))
myans.tail()

22


,Question ID,Options Selected
183,640653386564,"6406531285618,6406531285619"
184,640653386565,25
185,640653386566,0
186,640653386568,6406531285626
187,640653386569,0.40


In [78]:

# def replace_text(content, replacements = dict()):
#     lines = content.splitlines()

#     result = ""
#     in_text = False
#     for line in lines:
#         if line == "BT":
#             in_text = True

#         elif line == "ET":
#             in_text = False

#         elif in_text:
#             cmd = line[-2:]
#             if cmd.lower() == 'tj':
#                 replaced_line = line
#                 for k, v in replacements.items():
#                     replaced_line = replaced_line.replace(k, v)
#                 result += replaced_line + "\n"
#             else:
#                 result += line + "\n"
#     return result


# def process_data(object, replacements):
#     data = object.getData()
#     decoded_data = data.decode("utf-8")
#     replaced_data = replace_text(decoded_data, replacements)

#     encoded_data = replaced_data.encode("utf-8")
    
#     if object.decodedSelf is not None:
#         object.decodedSelf.setData(encoded_data)
#     else:
#         object.setData(encoded_data)




# # Provide replacements list that you need here
# replacements = { 'IITM': '<002c00510047004c004400510003002c005100560057004c00570058005700480003005200490003003700480046004b00510052004f0052004a005c000f0003003000440047005500440056000300100003002600480051005700550048>'}
# pdf = PdfFileReader("IIT M QUIZ 1 FOUNDATION DAD DIPLOMA QPD1.pdf")
# writer = PdfFileWriter()

# for page_number in range(0, pdf.getNumPages()):

#     page = pdf.getPage(page_number)
#     contents = page.getContents()

#     if isinstance(contents, DecodedStreamObject) or isinstance(contents, EncodedStreamObject):
#         process_data(contents, replacements)
#     elif len(contents) > 0:
#         for obj in contents:
#             if isinstance(obj, DecodedStreamObject) or isinstance(obj, EncodedStreamObject):
#                 streamObj = obj.getObject()
#                 process_data(streamObj, replacements)
#     page[NameObject("/Contents")] = contents.decodedSelf
#     writer.addPage(page)

# with open("result.pdf", 'wb') as out_file:
#     writer.write(out_file)


# pdf = PdfFileReader(filename)
# pdf.pages[1].getContents().get_data().decode().split("\n")
# # for page_number in range(0, pdf.getNumPages()):
# p=False
# pdf_content=""
# for i in range(0,pdf.getNumPages()):
#     page = pdf.getPage(i)
#     contents = page.extract_text()
#     if myans[my_question].values[0] in contents:
#         sp=i
#         p=True
#     if p:
#         pdf_content+=pdf.pages[i].getContents().getData().decode()
#     if myans[my_question].values[-1] in contents:
#         ep=i
#         p=False
#         break
# if i+1<pdf.getNumPages():
#     pdf_content+=pdf.pages[i+1].getContents().getData().decode()

In [79]:
def gentextarr(content):
    """
    Returns an array of text block properties of pdf
    Parameter:
        content: string object of page content obtained by page.getContents().getData().decode() 
    """
    lines = content.splitlines()
    result = []
    in_text = False
    for line in lines:
        if line == "BT":
            in_text = True
            temp=[]

        elif line == "ET":
            in_text = False
            result.append(temp)
            temp=[]

        elif in_text:
            if line[-2:].lower()=="tj" or line[-2:].lower()=="tf" or line[-2:].lower()=="rg":
                temp.append(line[:])
    return result

def gen_cmap(font,pg):
    """
    Returns a character map dictionary of a pdf 'filename' page 'pg' and font 
    Parameter:
     font: font key name in meta properties of pdf.
     pg: int page number of pdf
    """
    # pdf = PdfFileReader(filename)
    # page = pdf.getPage(pg)
    page=pg
    temp=False
    cmap={}
    for i in (page.getObject()['/Resources']['/Font'][font]['/ToUnicode'].getObject().getData().decode().split('\n')):
        if "endbfrange" in i:
            temp=False
        if temp:
            cmap[i[1:5]]=i[-5:-1]
        if "beginbfrange" in i:
            temp=True
    return cmap

def font_decoder(text,cmap):
    """
    Returns decoded word from pdf text hex code. 
    """

    count=0
    word=""
    temp=""
    for i in text:
        count+=1
        if count==4:
            temp+=i
            word+=chr(int(cmap.get(temp.lower(),temp.lower()),16))
            temp=""
            count=0
        else:
            temp+=i
    return word


In [80]:
pdf = PdfFileReader(filename)

In [81]:
cmapd={}
for i in range(pdf.getNumPages()):
    for k in pdf.pages[i].getObject()['/Resources']['/Font']:
        cmap=gen_cmap(k,pdf.pages[i])
        if cmap not in cmapd.values():
            cmapd[k+str(i)]=cmap

## PDF Content

In [82]:

sum=0
df=[[file_qid,"Correct Marks","Answers"]]
m_bool=False
marks=""
ans_bool=False
ans=""
row=[]
temp=[]
s_head=[]
temp_head=""
ishead=False
for p in range(0,pdf.getNumPages()):
    content=pdf.pages[p].getContents()
    if type(content)==ArrayObject:
        m=""
        for n in content:
            m+=n.getObject().getData().decode()
        content=m
    elif type(content)==EncodedStreamObject or type(content)==DecodedStreamObject:
        content=content.getData().decode()
    for i in gentextarr(content):
        text=""
        ans_bool=False
        for j in i:
            
            if j[-2:].lower()=="tf":
                cmap=gen_cmap(j.split()[0],pdf.pages[p])
                #checking for subject headings with font size =18
                if j.split()[1]=="18":
                    ishead=True
                    
            if j=="0 0.50196 0 rg" and len(row)>1:
                ans_bool=True
                
            elif j[-2:].lower()=="tj":
                text=font_decoder(j[1:-3],cmap)
                if ishead:
                    ishead=False
                    temp_head=text
                if (not m_bool) and (not ans_bool):
                    # print(text)
                    if str(file_qid) in text:
                        if len(temp)>0:
                            row.append(temp)
                            df.append(row)
                        row=[]
                        temp=[]
                        if text.split()[7].isnumeric():
                            s_head.append(temp_head)
                            row=[text.split()[7]]
                            m_bool=True
                    

                elif m_bool:
                    if "Correct Marks" in text:
                        # appending to heading list if marks is 0
                        # if text.split()[-1]=="0":
                        #     s_head.append(temp_head)
                        if len(row)==1:
                            row.append(text.split()[-1])
                        m_bool=False
                elif ans_bool:
                    if text!=" ":
                        if len(text)>=13 and text[:13].isnumeric():
                            temp.append(text[:13])
                        elif len(text.split())==3 and "to" in text.split()[1]:
                            t=text.split()
                            temp=[float(t[0]),float(t[-1])]
                        elif len(temp)==0:
                            temp.append(text)
if len(row)==3:
    df.append(row)

In [83]:
print(len(df))
df[1]

255


['640653386369', '0', ['6406531285046']]

In [84]:
df2=pd.DataFrame(df[1:],columns=df[0])
df2[file_qid]=df2[file_qid].apply(int)
#has some issues with index having one extra label
df2["Subjects"]=np.array(s_head)[0:-1]

df2=df2.sort_values([file_qid])
df2.head()

,Question Id,Correct Marks,Answers,Subjects
0,640653386369,0,[6406531285046],Maths2
4,640653386371,2,[-4],Maths2
5,640653386372,2,[0],Maths2
1,640653386373,2,"[6406531285051, 6406531285053]",Maths2
2,640653386374,2,[1],Maths2


In [85]:
df2=df2[df2[file_qid].isin(myans[my_question].values)].reset_index().drop("index",axis=1)
df2.head()

,Question Id,Correct Marks,Answers,Subjects
0,640653386545,0,[6406531285580],MLT
1,640653386546,6,[6406531285585],MLT
2,640653386547,6,[6406531285587],MLT
3,640653386548,6,[6406531285589],MLT
4,640653386549,5,[6406531285593],MLT


In [86]:
df2["My answer"]=myans.reset_index()[my_answer]
df2["My qid"]=myans.reset_index()[my_question]
print(len(df2))
df2

22


,Question Id,Correct Marks,Answers,Subjects,My answer,My qid
0,640653386545,0,[6406531285580],MLT,6406531285580,640653386545
1,640653386546,6,[6406531285585],MLT,6406531285585,640653386546
2,640653386547,6,[6406531285587],MLT,6406531285587,640653386547
3,640653386548,6,[6406531285589],MLT,6406531285589,640653386548
4,640653386549,5,[6406531285593],MLT,6406531285593,640653386549
5,640653386550,6,[6406531285597],MLT,6406531285597,640653386550
6,640653386551,5,[6406531285603],MLT,6406531285603,640653386551
7,640653386552,4,"[6406531285604, 6406531285606]",MLT,"6406531285604,6406531285606",640653386552
8,640653386553,5,"[0.19, 0.21]",MLT,0.20,640653386553
9,640653386554,5,[7],MLT,7,640653386554


In [87]:
def evaluate(df2):
#     df2['Score'] = df2.apply(lambda x: float(x['Correct Marks'])\
#                              if len(x["My answer"])==13 and x['My answer'] in x["Answers"]\
#                              else x["Score"], axis=1)
    
    df2['Score'] = df2.apply(lambda x: (len(set(x["My answer"].split(","))\
                                            .intersection(set(x["Answers"])))/len(x["Answers"]))*float(x['Correct Marks'])\
                             if type(x["Answers"][0])==str and len(x["Answers"][0])==13 and\
                                len(set(x["My answer"].split(",")))<=len(set(x["Answers"])) and\
                                len(set(x["My answer"].split(",")).intersection(set(x["Answers"])))>0\
                             else x["Score"], axis=1)
    
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks'])\
                             if len(x["My answer"])<13 and len(x['Answers'])>1 and\
                                float(x['My answer']) >= float(x["Answers"][0]) and\
                                float(x['My answer']) <= float(x["Answers"][1])\
                             else x["Score"], axis=1)
    
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks'])\
                             if len(x["My answer"])<13 and len(x['Answers'])==1 and\
                                float(x['My answer']) == float(x["Answers"][0])\
                             else x["Score"], axis=1)
    return df2

In [88]:
df2['Score']=np.zeros(df2.shape[0])
df2=evaluate(df2)
df2

,Question Id,Correct Marks,Answers,Subjects,My answer,My qid,Score
0,640653386545,0,[6406531285580],MLT,6406531285580,640653386545,0.0
1,640653386546,6,[6406531285585],MLT,6406531285585,640653386546,6.0
2,640653386547,6,[6406531285587],MLT,6406531285587,640653386547,6.0
3,640653386548,6,[6406531285589],MLT,6406531285589,640653386548,6.0
4,640653386549,5,[6406531285593],MLT,6406531285593,640653386549,5.0
5,640653386550,6,[6406531285597],MLT,6406531285597,640653386550,6.0
6,640653386551,5,[6406531285603],MLT,6406531285603,640653386551,5.0
7,640653386552,4,"[6406531285604, 6406531285606]",MLT,"6406531285604,6406531285606",640653386552,4.0
8,640653386553,5,"[0.19, 0.21]",MLT,0.20,640653386553,5.0
9,640653386554,5,[7],MLT,7,640653386554,5.0


## Final score

In [89]:
df2["Score"].apply(float).sum()

87.0

### Method 1: using index of 0 correct marks

In [109]:
s_index=df2[df2["Correct Marks"]=="0"].index
s_index

Int64Index([], dtype='int64')

In [110]:
subjects={}
for i in range(len(s_index)):
    if i==len(s_index)-1:
        subjects[df2["Subjects"][s_index[i]]]=df2.iloc[s_index[i]:]
    else:
        subjects[df2["Subjects"][s_index[i]]]=df2.iloc[s_index[i]:s_index[i+1]]
subjects.keys()

dict_keys([])

In [111]:
reports=[]
for i in subjects.values():
    print(i["Correct Marks"].apply(float).values)
    reports.append([i["Subjects"].unique()[0],i["Correct Marks"].apply(float).values.sum(),i["Score"].sum()])

In [112]:
for i in reports:
    print(i[0],":",i[2]/i[1]*100,"%")

### Method 2: Using groupby function

In [113]:
df2["Correct Marks"]=df2["Correct Marks"].apply(float)

In [114]:
report2=pd.DataFrame(df2.groupby("Subjects")[["Correct Marks","Score"]].sum())

In [115]:
report2["Percentage"]=(report2["Score"]*100/report2["Correct Marks"]).round(2)

In [116]:
report2.columns=["Total Marks","Scored","Percentage"]

In [117]:
report2

,Total Marks,Scored,Percentage
Subjects,,,
MLT,100.0,87.0,87.0
